# Analysing the Spread of Covid-19 in India


#### Importing Relevant Packages 

In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
print ('Modules are imported.')


Modules are imported.


#### Importing Relevant Datasets

In [2]:
#Daily cases nationally and by state; time-series - from Covid19India.com downloaded 1st Sept 2020
daily_national = pd.read_csv("datasets/India/national_case_time_series.csv")
daily_state = pd.read_csv("datasets/India/state_wise_overtime.csv")

In [3]:
#Global data on a country level for India up to 28/08/2020 - may be reduntant later given above but use to cross-check
covid_dataset_csv = pd.read_csv("datasets/time_series_covid19_confirmed_global.csv")

In [4]:
#Situation today state wise -  from Covid19India.com downloaded 1st Sept 2020
state_wise_cumulative = pd.read_csv("datasets/India/state_wise_today.csv")

In [5]:
#Dataset displaying the number of hospital beds in India by State - from Kaggle, bookmarked 'Data-Source'
hospital_beds = pd.read_csv('datasets/India/hospital_beds_india.csv')

In [6]:
#Dataset displaying Indian population - proxied on the number of Aadhar cards as the census is dated to 2011 by state 
# --> from Github
population = pd.read_csv('datasets/India/Indian_population_Aadhar.csv')

In [7]:
#Dataset displaying testing by state - from Covid19India.com downloaded 5th Sept
testing = pd.read_csv('datasets/India/statewise_tested_numbers_data.csv')
#Dataset displaying testing on a national level - from Covid19India.com downloaded 17th Oct
testing_country = pd.read_csv('datasets/India/tested_numbers_icmr_data (1).csv')

#### Exploring & Cleaning Datasets

##### 1)  Timeseries datasets:


In [8]:
#Datasets worked with:
#1. daily_national
#2. daily_state
#3. covid_dataset_csv

##### 1A) daily_national -->  national_ts_clean

In [9]:
daily_national.tail(10)

,Date,Daily Confirmed,Total Confirmed,Daily Recovered,Total Recovered,Daily Deceased,Total Deceased
205,22 August,70067,3043450,59101,2279893,918,56858
206,23 August,61749,3105199,56896,2336789,846,57704
207,24 August,59696,3164895,66305,2403094,854,58558
208,25 August,66873,3231768,64151,2467245,1066,59624
209,26 August,75995,3307763,56191,2523436,1017,60641
210,27 August,76827,3384590,59620,2583056,1066,61707
211,28 August,76657,3461247,64475,2647531,1019,62726
212,29 August,78479,3539726,64982,2712513,943,63669
213,30 August,79461,3619187,60422,2772935,960,64629
214,31 August,68766,3687953,64435,2837370,816,65445


In [10]:
daily_national.shape #215 days in the dataset; 30 January --> 31 August

(215, 7)

In [11]:
daily_national['Year'] = '2020'
daily_national['Date_W_Year'] = daily_national['Date'] + daily_national['Year'] 

In [12]:
daily_national['Date_W_Year'] = pd.to_datetime(daily_national.Date_W_Year)
national_ts_clean = daily_national.drop(['Date','Year'], axis =1)
national_ts_clean = national_ts_clean[['Date_W_Year', 'Daily Confirmed', 'Total Confirmed', 'Daily Recovered', 'Total Recovered', 'Daily Deceased', 'Total Deceased']]
national_ts_clean.rename(columns={'Date_W_Year':'Date'},inplace=True)

In [13]:
national_ts_clean.head(2)

,Date,Daily Confirmed,Total Confirmed,Daily Recovered,Total Recovered,Daily Deceased,Total Deceased
0,2020-01-30,1,1,0,0,0,0
1,2020-01-31,0,1,0,0,0,0


##### 1B) daily_state --> Cases_State, Deaths_State, Recov_State

In [14]:
daily_state.tail(3) 

,Date,Status,TT,AN,AP,AR,AS,BR,CH,CT,...,PB,RJ,SK,TN,TG,TR,UP,UT,WB,UN
510,31-Aug-20,Confirmed,68766,28,10004,78,3266,1324,191,1411,...,1466,1466,25,5956,1873,321,4782,592,2993,0
511,31-Aug-20,Recovered,64435,61,8772,63,1531,2267,135,686,...,1280,1719,5,6008,1849,201,4597,604,3318,0
512,31-Aug-20,Deceased,816,1,85,0,10,6,4,8,...,49,13,0,91,9,5,63,12,52,0


In [15]:
daily_state.shape #513 / 3; 171 days in the dataset; 14 March - 31 August

(513, 41)

In [16]:
#Dictionary to map onto new dataset columns 
state_dictionary = {'TT':'Total', 'AN':'Andaman and Nicobar Islands', 'AP': 'Andhra Pradesh', 'AR': 'Arunachal Pradesh', 'AS':'Assam', 'BR':'Bihar', 'CH':'Chandigarh', 'CT':'Chhattisgarh', 'DN':'Dadra and Nagar Haveli', 'DD':'Daman and Diu',
       'DL':'Delhi', 'GA':'Goa', 'GJ':'Gujarat', 'HR':'Haryana', 'HP':'Himachal Pradesh', 'JK':'Jammu and Kashmir', 'JH':'Jharkhand', 'KA':'Karnataka', 'KL':'Kerala', 'LA':'Ladakh', 'LD':'Lakshadweep', 'MP':'Madhya Pradesh',
       'MH':'Maharashtra', 'MN':'Manipur', 'ML':'Meghalaya', 'MZ':'Mizoram', 'NL':'Nagaland', 'OR':'Odisha', 'PY':'Puducherry', 'PB':'Punjab', 'RJ':'Rajasthan', 'SK':'Sikkim', 'TN':'Tamil Nadu', 'TG':'Telangana',
       'TR':'Tripura', 'UP':'Uttar Pradesh', 'UT':'Uttarakhand', 'WB':'West Bengal', 'UN':'State Unassigned'}

In [112]:
daily_state.rename(columns=state_dictionary, inplace=True)
daily_state['Date'] = pd.to_datetime(daily_state.Date)
daily_state.head()

,Date,Status,Total,Andaman and Nicobar Islands,Andhra Pradesh,Arunachal Pradesh,Assam,Bihar,Chandigarh,Chhattisgarh,...,Punjab,Rajasthan,Sikkim,Tamil Nadu,Telangana,Tripura,Uttar Pradesh,Uttarakhand,West Bengal,State Unassigned
0,2020-03-14,Confirmed,81,0,1,0,0,0,0,0,...,1,3,0,1,1,0,12,0,0,0
1,2020-03-14,Recovered,9,0,0,0,0,0,0,0,...,0,1,0,0,0,0,4,0,0,0
2,2020-03-14,Deceased,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2020-03-15,Confirmed,27,0,0,0,0,0,0,0,...,0,1,0,0,2,0,1,0,0,0
4,2020-03-15,Recovered,4,0,0,0,0,0,0,0,...,0,2,0,0,1,0,0,0,0,0


In [17]:
#Splitting the dataset into 3 
for Status in daily_state['Status'].unique():
        locals()['df_Statewise_' + Status] = daily_state[(daily_state.Status == Status)]

In [117]:
#Checking the resulting data is of the same shape
df_Statewise_Recovered.shape, df_Statewise_Confirmed.shape, df_Statewise_Deceased.shape

((171, 41), (171, 41), (171, 41))

##### 1C) covid_dataset_csv --> covid_aggregated

In [18]:
covid_dataset_csv.head(2) #Aggregate to get rid of region & call India to get national time-series later 

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,37894,37953,37999,38054,38070,38113,38129,38140,38143,38162
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,8119,8275,8427,8605,8759,8927,9083,9195,9279,9380


In [19]:
covid_dataset_csv.drop(["Lat","Long"], axis =1, inplace = True)
covid_aggregated = covid_dataset_csv.groupby("Country/Region").sum()
covid_aggregated.columns = pd.to_datetime(covid_aggregated.columns)

In [20]:
covid_aggregated.head(2) #January 22nd --> 30th August 2020 

,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,...,2020-08-21,2020-08-22,2020-08-23,2020-08-24,2020-08-25,2020-08-26,2020-08-27,2020-08-28,2020-08-29,2020-08-30
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,37894,37953,37999,38054,38070,38113,38129,38140,38143,38162
Albania,0,0,0,0,0,0,0,0,0,0,...,8119,8275,8427,8605,8759,8927,9083,9195,9279,9380


In [21]:
global_india = pd.DataFrame(covid_aggregated.loc['India'])

##### 2) Cross-Sectional State Wise Data (Cumulative)

In [22]:
state_wise_cumulative.head()
#Get rid of time only keep date in DD/MM/YYYY, delete total? 
#Active = Confirmed MINUS Recovered, Deaths & Migrated_Other

,State,Confirmed,Recovered,Deaths,Active,Last_Updated_Time,Migrated_Other,State_code,Delta_Confirmed,Delta_Recovered,Delta_Deaths,State_Notes
0,Total,3742022,2879366,66080,795987,01/09/2020 20:55:22,589,TT,54082,41989,647,NaN
1,Maharashtra,792541,573559,24583,194056,31/08/2020 21:54:22,343,MH,0,0,0,343 cases are marked as non-covid deaths in MH...
2,Tamil Nadu,433969,374172,7418,52379,01/09/2020 19:29:23,0,TN,5928,6031,96,[July 22]: 444 backdated deceased entries adde...
3,Andhra Pradesh,445139,339876,4053,101210,01/09/2020 19:29:25,0,AP,10368,9350,84,Total includes patients from other states and ...
4,Karnataka,351481,254626,5837,90999,01/09/2020 19:16:24,19,KA,9058,5159,135,NaN


In [23]:
state_wise_cumulative.shape

(38, 12)

In [24]:
state_wise_cumulative.set_index(['State'],inplace=True)

In [25]:
state_cumulative = state_wise_cumulative.drop(['Delta_Confirmed','Delta_Recovered','Delta_Deaths'],axis=1)

In [26]:
state_cumulative.head(2)

,Confirmed,Recovered,Deaths,Active,Last_Updated_Time,Migrated_Other,State_code,State_Notes
State,,,,,,,,
Total,3742022,2879366,66080,795987,01/09/2020 20:55:22,589,TT,NaN
Maharashtra,792541,573559,24583,194056,31/08/2020 21:54:22,343,MH,343 cases are marked as non-covid deaths in MH...


##### 3) Hospital Beds Data

In [27]:
hospital_beds.head()

,Sno,State/UT,NumPrimaryHealthCenters_HMIS,NumCommunityHealthCenters_HMIS,NumSubDistrictHospitals_HMIS,NumDistrictHospitals_HMIS,TotalPublicHealthFacilities_HMIS,NumPublicBeds_HMIS,NumRuralHospitals_NHP18,NumRuralBeds_NHP18,NumUrbanHospitals_NHP18,NumUrbanBeds_NHP18
0,1,Andaman & Nicobar Islands,27,4,NaN,3,34,1246,27,575,3,500
1,2,Andhra Pradesh,1417,198,31.0,20,1666,60799,193,6480,65,16658
2,3,Arunachal Pradesh,122,62,NaN,15,199,2320,208,2136,10,268
3,4,Assam,1007,166,14.0,33,1220,19115,1176,10944,50,6198
4,5,Bihar,2007,63,33.0,43,2146,17796,930,6083,103,5936


In [28]:
hospital_beds.drop(['Sno'],axis=1,inplace=True)
hospital_beds.set_index(['State/UT'], inplace=True)

In [29]:
hospital_beds.head(5)

,NumPrimaryHealthCenters_HMIS,NumCommunityHealthCenters_HMIS,NumSubDistrictHospitals_HMIS,NumDistrictHospitals_HMIS,TotalPublicHealthFacilities_HMIS,NumPublicBeds_HMIS,NumRuralHospitals_NHP18,NumRuralBeds_NHP18,NumUrbanHospitals_NHP18,NumUrbanBeds_NHP18
State/UT,,,,,,,,,,
Andaman & Nicobar Islands,27,4,NaN,3,34,1246,27,575,3,500
Andhra Pradesh,1417,198,31.0,20,1666,60799,193,6480,65,16658
Arunachal Pradesh,122,62,NaN,15,199,2320,208,2136,10,268
Assam,1007,166,14.0,33,1220,19115,1176,10944,50,6198
Bihar,2007,63,33.0,43,2146,17796,930,6083,103,5936


In [30]:
list(hospital_beds.columns)

['NumPrimaryHealthCenters_HMIS',
 'NumCommunityHealthCenters_HMIS',
 'NumSubDistrictHospitals_HMIS',
 'NumDistrictHospitals_HMIS',
 'TotalPublicHealthFacilities_HMIS',
 'NumPublicBeds_HMIS',
 'NumRuralHospitals_NHP18',
 'NumRuralBeds_NHP18',
 'NumUrbanHospitals_NHP18',
 'NumUrbanBeds_NHP18']

In [31]:
Beds_state = hospital_beds.drop(['NumPrimaryHealthCenters_HMIS',
 'NumCommunityHealthCenters_HMIS','NumSubDistrictHospitals_HMIS','NumDistrictHospitals_HMIS','TotalPublicHealthFacilities_HMIS','NumRuralHospitals_NHP18',
'NumUrbanHospitals_NHP18'],axis=1)

In [32]:
Beds_state['Total No. of Beds'] = Beds_state['NumPublicBeds_HMIS'] + Beds_state['NumRuralBeds_NHP18'] + Beds_state['NumUrbanBeds_NHP18']

In [33]:
Beds_state.head() #Assuming all 3 columns are independent counts so the Total is not double-counting!
#Also assuming these are ONLY PUBLIC facilities 

,NumPublicBeds_HMIS,NumRuralBeds_NHP18,NumUrbanBeds_NHP18,Total No. of Beds
State/UT,,,,
Andaman & Nicobar Islands,1246,575,500,2321
Andhra Pradesh,60799,6480,16658,83937
Arunachal Pradesh,2320,2136,268,4724
Assam,19115,10944,6198,36257
Bihar,17796,6083,5936,29815


In [34]:
Beds_state.rename(columns={'NumPublicBeds_HMIS':'No. of Public Beds', 'NumRuralBeds_NHP18':'No. of Beds in Rural Areas','NumUrbanBeds_NHP18':'No. of Beds in Urban Areas'},inplace=True)
Beds_state.head(4)

,No. of Public Beds,No. of Beds in Rural Areas,No. of Beds in Urban Areas,Total No. of Beds
State/UT,,,,
Andaman & Nicobar Islands,1246,575,500,2321
Andhra Pradesh,60799,6480,16658,83937
Arunachal Pradesh,2320,2136,268,4724
Assam,19115,10944,6198,36257


##### 4) Population Data based on Aadhar

In [35]:
population.head()

,State,Aadhaar assigned as of 2019,Area (per sq km)
0,Delhi,21763471,1483
1,Haryana,28941133,44212
2,Kerala,36475649,38852
3,Himachal Pradesh,7560770,55673
4,Punjab,30355185,50362


In [36]:
population.set_index(['State'],inplace=True)

In [37]:
population.rename(columns={'Area (per sq km)': 'Area (sq km)','Aadhaar assigned as of 2019':'Population (based on Aadhar)'},inplace=True)

In [38]:
population.head(2)

,Population (based on Aadhar),Area (sq km)
State,,
Delhi,21763471,1483
Haryana,28941133,44212


In [39]:
population['Population Density (Persons per sq km)'] = population['Population (based on Aadhar)']/ population['Area (sq km)']
population = population.round({'Population Density (Persons per sq km)':0})

In [40]:
#Calculate population weighting?

##### 5) Testing in India

In [8]:
#Split the state-wise data by state 
#Clean national then:
#Calculate difference from previous day to get moving average 
#Create a table of cumulative cases 

In [9]:
testing.tail(3)

,Updated On,State,RT PCR Tests,Antigen Tests,Other Tests,Total Tested,Tag (Total Tested),Positive,Negative,Unconfirmed,...,Num Calls State Helpline,Source1,Source2,Source3,Test positivity rate,Tests per thousand,Tests per million,Tests per positive case,Population NCP 2019 Projection,Unnamed: 31
4956,03/09/2020,West Bengal,NaN,NaN,NaN,2020784.0,Tested,NaN,NaN,NaN,...,NaN,https://www.wbhealth.gov.in/uploaded_files/cor...,https://www.wbhealth.gov.in/pages/corona/bulletin,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4957,04/09/2020,West Bengal,NaN,NaN,NaN,2066404.0,Tested,NaN,NaN,NaN,...,NaN,https://www.wbhealth.gov.in/uploaded_files/cor...,https://www.wbhealth.gov.in/pages/corona/bulletin,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4958,05/09/2020,West Bengal,NaN,NaN,NaN,2112185.0,Tested,NaN,NaN,NaN,...,NaN,https://www.wbhealth.gov.in/uploaded_files/cor...,https://www.wbhealth.gov.in/pages/corona/bulletin,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
testing.shape

(4959, 32)

In [11]:
#Create a state-by-state cumulative dataframe 

In [12]:
useless_cols = ['Cumulative People In Quarantine','Total People Currently in Quarantine', 'Tag (People in Quarantine)',
                'Total People Released From Quarantine', 'People in ICU','People on Ventilators','Num Isolation Beds',
                'Num ICU Beds', 'Num Ventilators','Total PPE','Total N95 Masks','Corona Enquiry Calls',
                'Num Calls State Helpline','Source1','Source2', 'Source3','Unnamed: 31']

In [13]:
testing_2 = testing.drop(useless_cols,axis=1)

In [14]:
testing_2.head(5)

,Updated On,State,RT PCR Tests,Antigen Tests,Other Tests,Total Tested,Tag (Total Tested),Positive,Negative,Unconfirmed,Test positivity rate,Tests per thousand,Tests per million,Tests per positive case,Population NCP 2019 Projection
0,17/04/2020,Andaman and Nicobar Islands,NaN,NaN,NaN,1403.0,Samples Sent,12.0,1210,181.0,0.86%,3.53,3534.0,117,397000.0
1,24/04/2020,Andaman and Nicobar Islands,NaN,NaN,NaN,2679.0,Samples Sent,27.0,NaN,246.0,1.01%,6.75,6748.0,99,397000.0
2,27/04/2020,Andaman and Nicobar Islands,NaN,NaN,NaN,2848.0,Samples Sent,33.0,NaN,106.0,1.16%,7.17,7174.0,86,397000.0
3,01/05/2020,Andaman and Nicobar Islands,NaN,NaN,NaN,3754.0,Samples Sent,33.0,NaN,199.0,0.88%,9.46,9456.0,114,397000.0
4,16/05/2020,Andaman and Nicobar Islands,NaN,NaN,NaN,6677.0,Samples Sent,33.0,NaN,136.0,0.49%,16.82,16819.0,202,397000.0


In [15]:
testing_2.rename(columns ={'Updated On':'Date'}, inplace=True)

In [162]:
print(testing_2.isnull().sum())

Date                                 0
State                                0
RT PCR Tests                      4734
Antigen Tests                     4751
Other Tests                       4942
Total Tested                        57
Tag (Total Tested)                  41
Positive                           654
Negative                          1976
Unconfirmed                       2837
Test positivity rate              1354
Tests per thousand                1328
Tests per million                 1328
Tests per positive case           1443
Population NCP 2019 Projection    1328
dtype: int64


In [17]:
testing_2.head(3)

,Date,State,RT PCR Tests,Antigen Tests,Other Tests,Total Tested,Tag (Total Tested),Positive,Negative,Unconfirmed,Test positivity rate,Tests per thousand,Tests per million,Tests per positive case,Population NCP 2019 Projection
0,17/04/2020,Andaman and Nicobar Islands,NaN,NaN,NaN,1403.0,Samples Sent,12.0,1210,181.0,0.86%,3.53,3534.0,117,397000.0
1,24/04/2020,Andaman and Nicobar Islands,NaN,NaN,NaN,2679.0,Samples Sent,27.0,NaN,246.0,1.01%,6.75,6748.0,99,397000.0
2,27/04/2020,Andaman and Nicobar Islands,NaN,NaN,NaN,2848.0,Samples Sent,33.0,NaN,106.0,1.16%,7.17,7174.0,86,397000.0


In [19]:
testing_2['State'] = testing_2['State'].str.replace(' and ', ' ')
testing_2['State'].unique() 

array(['Andaman Nicobar Islands', 'Andhra Pradesh', 'Arunachal Pradesh',
       'Assam', 'Bihar', 'Chandigarh', 'Chhattisgarh',
       'Dadra Nagar Haveli Daman Diu', 'Delhi', 'Goa', 'Gujarat',
       'Haryana', 'Himachal Pradesh', 'Jammu Kashmir', 'Jharkhand',
       'Karnataka', 'Kerala', 'Ladakh', 'Madhya Pradesh', 'Maharashtra',
       'Manipur', 'Meghalaya', 'Mizoram', 'Nagaland', 'Odisha',
       'Puducherry', 'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu',
       'Telangana', 'Tripura', 'Uttar Pradesh', 'Uttarakhand',
       'West Bengal'], dtype=object)

In [20]:
for State in testing_2['State'].unique():
    try: 
        locals()['df_' + State.split(' ')[0] + State.split(' ')[1][0]] =  testing_2[(testing_2.State == State)]
    except IndexError: 
        locals()['df_' + State] = testing_2[(testing_2.State == State)]

In [75]:
#Creating a cumulative table:
testing_cross_sect = testing_2[testing_2.Date == '01/09/2020']

In [76]:
testing_cross_sect.shape

(35, 15)

In [69]:
testing_cross_sect.set_index('State', inplace=True)

In [83]:
testing_cross_sect.head() #Showing total finals; active, negative, tested 

,Date,State,RT PCR Tests,Antigen Tests,Other Tests,Total Tested,Tag (Total Tested),Positive,Negative,Unconfirmed,Test positivity rate,Tests per thousand,Tests per million,Tests per positive case,Population NCP 2019 Projection
110,01/09/2020,Andaman and Nicobar Islands,NaN,NaN,NaN,35104.0,Samples Sent,3160.0,NaN,246.0,NaN,NaN,NaN,NaN,NaN
260,01/09/2020,Andhra Pradesh,NaN,NaN,NaN,3782746.0,Tested,NaN,3337607,NaN,NaN,NaN,NaN,NaN,NaN
400,01/09/2020,Arunachal Pradesh,NaN,NaN,NaN,170445.0,Samples Collected,NaN,160911,27.0,NaN,NaN,NaN,NaN,NaN
535,01/09/2020,Assam,NaN,NaN,NaN,2299893.0,Tested,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
687,01/09/2020,Bihar,NaN,NaN,NaN,3302720.0,Tested,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [134]:
testing_country.head(5)

,Update Time Stamp,Tested As Of,Daily RTPCR Samples Collected_ICMR Application,Total RTPCR Samples Collected_ICMR Application,Total Samples Tested,Total Individuals Tested,Total Positive Cases,Tests conducted by Private Labs,Sample Reported today,Positive cases from samples reported,Source,Source 1,Source 3,Test positivity rate,Individuals Tested Per Confirmed Case,Tests Per Confirmed Case,Tests per million
0,13/03/2020 00:00:00,13/03/2020,NaN,NaN,6500.0,5900.0,78,NaN,NaN,NaN,Press_Release_ICMR_13March2020.pdf,NaN,NaN,1.20%,75.64,83.33,5.0
1,18/03/2020 18:00:00,18/03/2020,NaN,NaN,13125.0,12235.0,150,NaN,NaN,NaN,ICMR_website_update_18March_6PM_IST.pdf,NaN,NaN,1.14%,81.57,87.50,10.0
2,19/03/2020 10:00:00,19/03/2020,NaN,NaN,13316.0,12426.0,168,NaN,NaN,NaN,ICMR_website_update_19March_10AM_IST_V2.pdf,NaN,NaN,1.26%,73.96,79.26,10.0
3,19/03/2020 18:00:00,19/03/2020,NaN,NaN,14175.0,13285.0,182,NaN,NaN,NaN,ICMR_website_update_19March_6PM_IST.pdf,NaN,NaN,1.28%,72.99,77.88,11.0
4,20/03/2020 10:00:00,20/03/2020,NaN,NaN,14376.0,13486.0,206,NaN,NaN,NaN,ICMR_website_update_20March_10AM_IST.pdf,NaN,NaN,1.43%,65.47,69.79,11.0


In [135]:
testing_national = testing_country.drop(['Source', 'Source 1', 'Source 3'], axis=1)
testing_national.rename(columns ={'Tested As Of': 'Date'}, inplace = True)
testing_national.head(3)

,Update Time Stamp,Date,Daily RTPCR Samples Collected_ICMR Application,Total RTPCR Samples Collected_ICMR Application,Total Samples Tested,Total Individuals Tested,Total Positive Cases,Tests conducted by Private Labs,Sample Reported today,Positive cases from samples reported,Test positivity rate,Individuals Tested Per Confirmed Case,Tests Per Confirmed Case,Tests per million
0,13/03/2020 00:00:00,13/03/2020,NaN,NaN,6500.0,5900.0,78,NaN,NaN,NaN,1.20%,75.64,83.33,5.0
1,18/03/2020 18:00:00,18/03/2020,NaN,NaN,13125.0,12235.0,150,NaN,NaN,NaN,1.14%,81.57,87.50,10.0
2,19/03/2020 10:00:00,19/03/2020,NaN,NaN,13316.0,12426.0,168,NaN,NaN,NaN,1.26%,73.96,79.26,10.0


In [136]:
testing_national.shape #217 days since 13th March to 17th October (data downloaded) - there are some duplicates!

(221, 14)

In [137]:
#Spotting Duplicates:
df = pd.value_counts(testing_national.Date).to_frame().reset_index()
df.columns = ['Date','Frequency']
duplicates = list(df.loc[df['Frequency'] > 1, 'Date'])
len(duplicates) 
#There are 9 duplicates in the data implying that there are records for 207 days; a couple are thus also missing

9

In [138]:
#Show duplicate values from dataframe; notice the timestamp is different and 
#the values for the later timestamps are always larger
#So we can delete all of the 10am values and keep the EoD or 6pm values 
dupl_df = testing_national.loc[testing_national['Date'].isin(duplicates)]
dupl_df

,Update Time Stamp,Date,Daily RTPCR Samples Collected_ICMR Application,Total RTPCR Samples Collected_ICMR Application,Total Samples Tested,Total Individuals Tested,Total Positive Cases,Tests conducted by Private Labs,Sample Reported today,Positive cases from samples reported,Test positivity rate,Individuals Tested Per Confirmed Case,Tests Per Confirmed Case,Tests per million
2,19/03/2020 10:00:00,19/03/2020,NaN,NaN,13316.0,12426.0,168,NaN,NaN,NaN,1.26%,73.96,79.26,10.0
3,19/03/2020 18:00:00,19/03/2020,NaN,NaN,14175.0,13285.0,182,NaN,NaN,NaN,1.28%,72.99,77.88,11.0
4,20/03/2020 10:00:00,20/03/2020,NaN,NaN,14376.0,13486.0,206,NaN,NaN,NaN,1.43%,65.47,69.79,11.0
5,20/03/2020 18:00:00,20/03/2020,NaN,NaN,15404.0,14514.0,236,NaN,NaN,NaN,1.53%,61.50,65.27,12.0
6,21/03/2020 10:00:00,21/03/2020,NaN,NaN,15701.0,14811.0,271,NaN,NaN,NaN,1.73%,54.65,57.94,12.0
7,21/03/2020 18:00:00,21/03/2020,NaN,NaN,16911.0,16021.0,315,NaN,NaN,NaN,1.86%,50.86,53.69,13.0
8,22/03/2020 10:00:00,22/03/2020,NaN,NaN,16999.0,16109.0,341,NaN,NaN,NaN,2.01%,47.24,49.85,13.0
9,22/03/2020 18:00:00,22/03/2020,NaN,NaN,18127.0,17237.0,396,NaN,NaN,NaN,2.18%,43.53,45.78,14.0
10,23/03/2020 10:00:00,23/03/2020,NaN,NaN,18383.0,17493.0,415,NaN,NaN,NaN,2.26%,42.15,44.30,14.0
11,23/03/2020 20:00:00,23/03/2020,NaN,NaN,20707.0,19817.0,471,NaN,NaN,NaN,2.27%,42.07,43.96,16.0


In [149]:
#generating list of index items we want to remove: 
dupl_index = []
for i in dup_df.index: 
    if i%2 == 0:
        dupl_index.append(i)
print(dupl_index)

[2, 4, 6, 8, 10, 12, 14, 16, 22]


In [150]:
#Remove from dataframe and check shape; 9 items removed!
testing_national.drop(dupl_index, axis = 0, inplace = True)
testing_national.shape

(212, 14)

In [160]:
#identifying missing values; we can see that a more reliable measure is 'Sample Reported Today', 
#rather than 'Total Samples Tested'
print(testing_national.isnull().sum())

Update Time Stamp                                   0
Date                                                1
Daily RTPCR Samples Collected_ICMR Application    143
Total RTPCR Samples Collected_ICMR Application    143
Total Samples Tested                                2
Total Individuals Tested                          188
Total Positive Cases                              181
Tests conducted by Private Labs                   209
Sample Reported today                              45
Positive cases from samples reported              197
Test positivity rate                                2
Individuals Tested Per Confirmed Case             181
Tests Per Confirmed Case                          181
Tests per million                                   2
dtype: int64


#### Visualising Recovered, Deaths & Confirmed Cases

In [ ]:
#Objectives:
#let's try find top death rate
#let's try find top recovery rate too and see if it matches hospital capacity - trying to identify 'success stories'
#let's try to map general trends:
## 1 graph of cases daily for top 10, middle 10 and bottom 10 states
## 1 graph of national cases = daily_national showing the new cases, new conifrmed and deaths over time- differentiate

In [218]:
Cases_State.tail(10)

,Total,Andaman and Nicobar Islands,Andhra Pradesh,Arunachal Pradesh,Assam,Bihar,Chandigarh,Chhattisgarh,Dadra and Nagar Haveli,Daman and Diu,...,Punjab,Rajasthan,Sikkim,Tamil Nadu,Telangana,Tripura,Uttar Pradesh,Uttarakhand,West Bengal,State Unassigned
Date,,,,,,,,,,,,,,,,,,,,,
2020-08-22,70067,61,10276,97,1560,2238,145,704,40,0,...,1316,1310,45,5980,2474,280,5217,483,3232,0
2020-08-23,61749,52,7895,40,1272,2247,142,704,36,0,...,1136,1345,22,5975,2384,331,5325,558,3274,0
2020-08-24,59696,44,8601,49,1879,1227,117,1136,18,0,...,1505,1346,43,5967,1842,200,4601,405,2967,0
2020-08-25,66873,41,9927,100,1973,1444,174,1287,21,0,...,1293,1370,29,5951,2579,293,5006,485,2964,0
2020-08-26,75995,40,10830,143,2179,2163,167,1209,22,0,...,1513,1345,11,5958,3018,329,5640,535,2974,0
2020-08-27,76827,33,10621,78,2036,1860,188,1438,42,0,...,1746,1345,56,5981,2795,385,5391,728,2997,0
2020-08-28,76657,32,10526,112,2560,1998,160,1245,31,0,...,1542,1355,34,5996,2932,509,5405,588,2982,0
2020-08-29,78479,31,10548,132,2427,2087,261,1513,15,0,...,1470,1407,26,6352,2751,443,5633,706,3012,0
2020-08-30,79461,23,10603,157,1980,2078,170,1346,32,0,...,1678,1450,25,6495,2924,447,6175,664,3019,0


In [ ]:
#Visualising the top, middle and bottom 15 states, to find out which, we need to look at the cumulative dataset

In [227]:
state_cumulative.head()

,Confirmed,Recovered,Deaths,Active,Last_Updated_Time,Migrated_Other,State_code,State_Notes
State,,,,,,,,
Total,3742022,2879366,66080,795987,01/09/2020 20:55:22,589,TT,NaN
Maharashtra,792541,573559,24583,194056,31/08/2020 21:54:22,343,MH,343 cases are marked as non-covid deaths in MH...
Tamil Nadu,433969,374172,7418,52379,01/09/2020 19:29:23,0,TN,[July 22]: 444 backdated deceased entries adde...
Andhra Pradesh,445139,339876,4053,101210,01/09/2020 19:29:25,0,AP,Total includes patients from other states and ...
Karnataka,351481,254626,5837,90999,01/09/2020 19:16:24,19,KA,NaN


In [237]:
population.head()

,State,Aadhaar assigned as of 2019,Area (per sq km)
0,Delhi,21763471,1483
1,Haryana,28941133,44212
2,Kerala,36475649,38852
3,Himachal Pradesh,7560770,55673
4,Punjab,30355185,50362


In [243]:
population.head(2)

,State,Aadhaar assigned as of 2019,Area (per sq km)
0,Delhi,21763471,1483
1,Haryana,28941133,44212


In [12]:
state_cumulative.head(1)

,Confirmed,Recovered,Deaths,Active,Last_Updated_Time,Migrated_Other,State_code,State_Notes
State,,,,,,,,
Total,3742022,2879366,66080,795987,01/09/2020 20:55:22,589,TT,NaN


In [231]:
Top_confirmed = state_cumulative.sort_values(by='Confirmed', ascending=False)
Top_active = state_cumulative.sort_values(by='Active', ascending=False)
Top_deaths = state_cumulative.sort_values(by='Deaths', ascending=False) 
Top_recovered = state_cumulative.sort_values(by='Recovered', ascending=False) 